### Can we classify each phase as relatively low or high anxiety for each subject? ###
#### APD, WESAD ####

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\zhoux\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# metrics = train.Metrics.ALL
# metrics = train.Metrics.ECG \
    # + train.Metrics.EDA \
    # + train.Metrics.ANKLE + train.Metrics.WRIAAST

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_apd = [
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Break"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", 
        "Speech_Relax", "Speech_Anticipate"
    ],
    # [
    #     "Baseline_Rest", 
    #     "BugBox_Relax", "BugBox_Anticipate", "BugBox_Break",
    #     "Speech_Relax", "Speech_Anticipate", "Speech_Break"
    # ],
    ["BugBox_Break", "Speech_Break"],
    ["BugBox_Exposure", "Speech_Exposure"]
]

model_phases_wesad = [
    [
        dr_w.Phases.BASE,
        dr_w.Phases.FUN,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [
        dr_w.Phases.BASE,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [
        dr_w.Phases.BASE,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [
        dr_w.Phases.BASE,
    ]
]

apd_label_type = "Anxiety"
wesad_label_type = "stai"

models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    "KNN": KNeighborsClassifier(n_neighbors=7),
    "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

test_size = 0.8

`use_label_encoder` is deprecated in 1.7.0.


In [4]:
# TRAIN ON APD AND TEST ON WESAD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for i, phases_apd in enumerate(model_phases_apd):
    for j, phases_wesad in enumerate(model_phases_wesad):
        print(f"APD PHASES {i} " + "-"*50)
        print(f"WESAD PHASES {j} " + "-"*50)
        x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=apd_label_type)
        x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type)
        # drop subjects with noisy data
        x_a = x_a[x_a['subject'] != 84.0]
        x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature
        y_a = y_a[y_a['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]
        
        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        acc_results = {
            "SVM": [], "KNN": [],
            "DT": [],
            "LogReg": [],
            "Bayes": [],
            "XGB": []
        }
        reports = {
            "SVM": [], "KNN": [],
            "DT": [],
            "LogReg": [],
            "Bayes": [],
            "XGB": [],
        }
        num_iters = 10
        for _ in range(num_iters):
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=False, save_metrics=True, test_size=test_size)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.65:
                print(f"Model evaluation metrics for {model_name}:")
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("\n")

APD PHASES 0 --------------------------------------------------
WESAD PHASES 0 --------------------------------------------------
SVM accuracy over 10 rounds: 0.37142857142857144
KNN accuracy over 10 rounds: 0.37142857142857144
DT accuracy over 10 rounds: 0.5242857142857142
LogReg accuracy over 10 rounds: 0.37142857142857144
Bayes accuracy over 10 rounds: 0.45714285714285713
XGB accuracy over 10 rounds: 0.37142857142857144


APD PHASES 0 --------------------------------------------------
WESAD PHASES 1 --------------------------------------------------
SVM accuracy over 10 rounds: 0.39285714285714285
KNN accuracy over 10 rounds: 0.39285714285714285
DT accuracy over 10 rounds: 0.5767857142857143
LogReg accuracy over 10 rounds: 0.39285714285714285
Bayes accuracy over 10 rounds: 0.4107142857142857
XGB accuracy over 10 rounds: 0.42857142857142855


APD PHASES 0 --------------------------------------------------
WESAD PHASES 2 --------------------------------------------------
SVM accuracy 

In [5]:
# TRAIN ON WESAD AND TEST ON APD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for i, phases_wesad in enumerate(model_phases_wesad):
    for j, phases_apd in enumerate(model_phases_apd):
        print(f"WESAD PHASES {i} " + "-"*50)
        print(f"APD PHASES {j} " + "-"*50)
        x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type)
        x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=apd_label_type)
        # drop subjects with noisy data
        x_b = x_b[x_b['subject'] != 84.0]
        x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature
        y_b = y_b[y_b['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]
        
        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        acc_results = {
            "SVM": [], "KNN": [],
            "DT": [],
            "LogReg": [],
            "XGB": []
        }
        reports = {
            "SVM": [], "KNN": [],
            "DT": [],
            "LogReg": [],
            "XGB": [],
        }
        num_iters = 10
        for _ in range(num_iters):
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=False, save_metrics=True, test_size=test_size)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.65:
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Report:\nPrecision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("\n")

WESAD PHASES 0 --------------------------------------------------
APD PHASES 0 --------------------------------------------------
SVM accuracy over 10 rounds: 0.3244444444444444
KNN accuracy over 10 rounds: 0.3222222222222223
DT accuracy over 10 rounds: 0.4253333333333333
LogReg accuracy over 10 rounds: 0.47111111111111115
XGB accuracy over 10 rounds: 0.34222222222222226


WESAD PHASES 0 --------------------------------------------------
APD PHASES 1 --------------------------------------------------
SVM accuracy over 10 rounds: 0.6485714285714286
KNN accuracy over 10 rounds: 0.6485714285714286
DT accuracy over 10 rounds: 0.6471428571428571
LogReg accuracy over 10 rounds: 0.6485714285714286
XGB accuracy over 10 rounds: 0.6285714285714284


WESAD PHASES 0 --------------------------------------------------
APD PHASES 2 --------------------------------------------------
SVM accuracy over 10 rounds: 0.30399999999999994
KNN accuracy over 10 rounds: 0.29999999999999993
DT accuracy over 10 ro

In [110]:
# CLOSER ANALYSIS OF:
# APD 1, 20% AND WESAD 2, 100%
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


test_size = 0.9

apd_label_type = "Anxiety"
wesad_label_type = "stai"

model_phases_apd = [
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
    ]
]

model_phases_wesad = [
    [
        dr_w.Phases.BASE, dr_w.Phases.TSST, dr_w.Phases.MEDI_1, dr_w.Phases.MEDI_2
    ]
]

for i, phases_wesad in enumerate(model_phases_wesad):
    for j, phases_apd in enumerate(model_phases_apd):
        print(f"WESAD PHASES {i} " + "-"*50)
        print(f"APD PHASES {j} " + "-"*50)
        x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type)
        x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=apd_label_type)
        
        # drop phaseId label
        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # drop subjects with noisy data
        x_b = x_b[x_b['subject'] != 84.0]
        x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature
        y_b = y_b[y_b['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]

        # x_a = x_a.drop(["subject"], axis=1)
        # x_b = x_b.drop(["subject"], axis=1)

        # 0-1 scaling
        for c in range(2, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(2, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col
            
        # print(x_a.head())
        # print(x_b.head())

        acc_results = {
            "SVM": [],
            "XGB": []
        }
        reports = {
            "SVM": [],
            "XGB": [],
        }
        shap_values = {
            "SVM": [],
            "XGB": [],
        }
        num_iters = 10
        for i in range(num_iters):
            if i % 3 == 0:
                get_shap_values = True
            else:
                get_shap_values = False
            get_shap_values = False
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=False, save_metrics=True, test_size=test_size, get_shap_values=get_shap_values)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])
                shap_values[model_name].append(out[model_name][2])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.65:
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Report:\nPrecision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}\n")
            for i in range(len(shap_values[model_name])):
                if shap_values[model_name][i] is not None:
                    shap.summary_plot(shap_values[model_name][i])
        print("\n")

WESAD PHASES 0 --------------------------------------------------
APD PHASES 0 --------------------------------------------------
SVM accuracy over 10 rounds: 0.4825415589237144
XGB accuracy over 10 rounds: 0.5949271719580131




In [21]:
# CLOSER ANALYSIS OF:
# APD 2, 100%, WESAD 3, 0%
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)

model_phases_apd = [
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Break"
    ]
]

model_phases_wesad = [
    [
        dr_w.Phases.BASE, dr_w.Phases.MEDI_1, dr_w.Phases.MEDI_2
    ]
]

test_size = 1.0

apd_label_type = "Anxiety"
wesad_label_type = "stai"

for i, phases_apd in enumerate(model_phases_apd):
    for j, phases_wesad in enumerate(model_phases_wesad):
        print(f"APD PHASES {i} " + "-"*50)
        print(f"WESAD PHASES {j} " + "-"*50)
        x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=apd_label_type)
        x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type)
        # drop subjects with noisy data
        x_a = x_a[x_a['subject'] != 84.0]
        x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature
        y_a = y_a[y_a['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]

        print(y_a.loc[:, "label"].value_counts() + y_b.loc[:, "label"].value_counts())
        
        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        acc_results = {
            "DT": [],
            "LogReg": [],
        }
        reports = {
            "DT": [],
            "LogReg": [],
        }
        shap_values = {
            "DT": [],
            "LogReg": [],
        }
        num_iters = 10
        for i in range(num_iters):
            if i % 3 == 0:
                get_shap_values = True
            else:
                get_shap_values = False
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=False, save_metrics=True, test_size=test_size, get_shap_values=get_shap_values)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])
                shap_values[model_name].append(out[model_name][2])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.65:
                print(f"Model evaluation metrics for {model_name}:")
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
            if shap_values[model_name][0] is not None:
                print("SHAP summary plot: ")
                shap.summary_plot(shap_values[model_name][0], x_a)
        print("\n")

APD PHASES 0 --------------------------------------------------
WESAD PHASES 0 --------------------------------------------------
0    146
1    253
Name: label, dtype: int64
DT accuracy over 10 rounds: 0.23809523809523808
LogReg accuracy over 10 rounds: 0.619047619047619




In [44]:
# CLOSER ANALYSIS OF:
# APD 2, 0%, WESAD 1, 100%
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)

model_phases_apd = [
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Break"
    ]
]

model_phases_wesad = [
    [
        dr_w.Phases.BASE,
        dr_w.Phases.FUN,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ]
]

test_size = 0.8

apd_label_type = "Anxiety"
wesad_label_type = "stai"

get_shap_values = False

for i, phases_wesad in enumerate(model_phases_wesad):
    for j, phases_apd in enumerate(model_phases_apd):
        print(f"WESAD PHASES {i} " + "-"*50)
        print(f"APD PHASES {j} " + "-"*50)
        x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type)
        x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=apd_label_type)
        # drop subjects with noisy data
        x_b = x_b[x_b['subject'] != 84.0]
        x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature
        y_b = y_b[y_b['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]

        print(y_a.loc[:, "label"].value_counts() + y_b.loc[:, "label"].value_counts())
        
        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        acc_results = {
            "KNN": [],
            "SVM": [],
            "LogReg": [],
        }
        reports = {
            "KNN": [],
            "SVM": [],
            "LogReg": [],
        }
        shap_values = {
            "KNN": [],
            "SVM": [],
            "LogReg": [],
        }
        num_iters = 10
        for _ in range(num_iters):
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=False, save_metrics=True, test_size=test_size)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])
                shap_values[model_name].append(out[model_name][2])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.65:
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Report:\nPrecision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
            if shap_values[model_name][0] is not None:
                print("SHAP summary plot: ")
                shap.summary_plot(shap_values[model_name][0], x_a)
        print("\n")

WESAD PHASES 0 --------------------------------------------------
APD PHASES 0 --------------------------------------------------
0    167
1    260
Name: label, dtype: int64
KNN accuracy over 10 rounds: 0.6032900752989011
SVM accuracy over 10 rounds: 0.6409086936107309
LogReg accuracy over 10 rounds: 0.6373178165337003


